In [2]:
%pip install pyarrow

Note: you may need to restart the kernel to use updated packages.


In [3]:
!python -V

Python 3.10.13


### Import and Read

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df_202301 = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
df_202302 = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')

### Q1. Downloading the data
For January 2023, how many columns are there?

In [6]:
answer_1 = len(df_202301.head().columns)
display(answer_1)

19

### Q2. Computing duration

Now let's compute the `duration` variable. It should contain the duration of a ride in minutes. 

What's the standard deviation of the trips duration in January?

In [7]:
df_202301.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [8]:
def add_duration_column_by_minutes(df):
    df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df.duration.apply(lambda td: td.total_seconds() / 60)
    return df

df_202301_with_duration = add_duration_column_by_minutes(df_202301)
answer_2 =df_202301_with_duration.duration.std()
display(answer_2)

42.59435124195458

###  Q3. Dropping outliers
Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers

In [9]:
def describe_duration(df):
    return df.duration.describe().apply(lambda x: format(x, 'f'))

def drop_outliers(df):
    # drop less than 1 min and  greater than 60 min
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    return df

In [10]:
# Observe outlier dropping and explore
display(describe_duration(df_202301))
df_202301_no_outliers = drop_outliers(df_202301_with_duration)
display(describe_duration(df_202301_no_outliers))
display(df_202301_no_outliers["duration"].value_counts())

count    3066766.000000
mean          15.668995
std           42.594351
min          -29.200000
25%            7.116667
50%           11.516667
75%           18.300000
max        10029.183333
Name: duration, dtype: object

count    3009173.000000
mean          14.204864
std            9.939386
min            1.000000
25%            7.216667
50%           11.550000
75%           18.183333
max           60.000000
Name: duration, dtype: object

duration
7.000000     3697
8.000000     3665
9.000000     3558
6.000000     3523
11.000000    3518
             ... 
59.916667      27
59.483333      26
59.333333      26
58.233333      26
59.783333      23
Name: count, Length: 3541, dtype: int64

In [11]:
answer_3 = df_202301_no_outliers.duration.count()/df_202301.duration.count()
display (answer_3)

0.9812202822125979

### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will 
  label encode them)
* Fit a dictionary vectorizer 
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [12]:
from sklearn.feature_extraction import DictVectorizer

In [14]:
def one_hot_encode(df):
    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    dv = DictVectorizer()
    
    X_train = dv.fit_transform(train_dicts)
    return X_train

/tmp/ipykernel_13401/2190387643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


516

In [ ]:
encoded_df = one_hot_encode(df_202301_no_outliers)
answer_4 = encoded_df.get_shape()[1]
display(answer_4)

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model. 

* Train a plain linear regression model with default parameters 
* Calculate the RMSE of the model on the training data

What's the RMSE on train?

* 3.64
* 7.64
* 11.64
* 16.64

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
def train_and_predict(df):
    df = add_duration_column_by_minutes(df)
    df = drop_outliers(df)
    X_train = one_hot_encode(df)
    
    target = 'duration'
    y_train = df[target].values

    lr = LinearRegression()
    lr.fit(encoded_df, y_train)

    y_pred = lr.predict(X_train)
    
    rmse = mean_squared_error(y_train, y_pred, squared=False)
    return rmse

## Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

What's the RMSE on validation?

* 3.81
* 7.81
* 11.81
* 16.81